In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import time
import dask
from dask.distributed import Client
import multiprocessing

In [2]:
!wget https://chip-atlas.dbcls.jp/data/metadata/experimentList.tab

--2023-03-01 22:04:15--  https://chip-atlas.dbcls.jp/data/metadata/experimentList.tab
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving chip-atlas.dbcls.jp (chip-atlas.dbcls.jp)... 133.39.78.111
Connecting to chip-atlas.dbcls.jp (chip-atlas.dbcls.jp)|133.39.78.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278349819 (265M) [application/octet-stream]
Saving to: ‘experimentList.tab’

experimentList.tab  100%[===================>] 265.45M   484KB/s    in 5m 44s  

2023-03-01 22:10:00 (790 KB/s) - ‘experimentList.tab’ saved [278349819/278349819]



In [41]:
df_tmp = pd.read_csv('experimentList.tab', sep='\t', lineterminator='\n', usecols=range(6), header=None)
df_tmp

,0,1,2,3,4,5
0,DRX184547,ce10,Input control,Input control,Larvae,L1
1,DRX184548,ce10,Histone,H3K4me2,Larvae,L1
2,DRX184549,ce10,Histone,H3K27me3,Larvae,L1
3,DRX184550,ce10,Histone,H3K36me3,Larvae,L1
4,DRX184551,ce10,Input control,Input control,Larvae,L1
...,...,...,...,...,...,...
675552,SRX957114,sacCer3,Bisulfite-Seq,Bisulfite-Seq,Yeast strain,BY4741
675553,SRX957115,sacCer3,Bisulfite-Seq,Bisulfite-Seq,Yeast strain,BY4741
675554,SRX957117,sacCer3,Bisulfite-Seq,Bisulfite-Seq,Yeast strain,BY4741
675555,SRX957118,sacCer3,Bisulfite-Seq,Bisulfite-Seq,Yeast strain,W303


In [42]:
df_tmp.to_csv('exp_edited.tab', header=False, index=False)

In [2]:
df = dd.read_csv('exp_edited.tab',
                sep = ',', 
                names = ['id', 'Genome assembly', 'Antigen class', 'Antigen', 'Cell type class', 'Cell type'],
                blocksize = '10mb'
                )

In [3]:
# NCORES = multiprocessing.cpu_count() - 8
NCORES = 2
client = Client(n_workers=NCORES, threads_per_worker=2)

In [4]:
part = df.partitions[0]
part.head()

,id,Genome assembly,Antigen class,Antigen,Cell type class,Cell type
0,DRX184547,ce10,Input control,Input control,Larvae,L1
1,DRX184548,ce10,Histone,H3K4me2,Larvae,L1
2,DRX184549,ce10,Histone,H3K27me3,Larvae,L1
3,DRX184550,ce10,Histone,H3K36me3,Larvae,L1
4,DRX184551,ce10,Input control,Input control,Larvae,L1


In [11]:
len(part)

231874

In [89]:
def compute_part(num, options):
    part = df.partitions[num]
    
    if options[0] != '':
        tmp = options[0].split(',')
        part = part.loc[part['id'].isin(tmp)]
    if options[1] != '':
        tmp = options[1].split(',')
        part = part.loc[part['Genome assembly'].isin(tmp)]
    if options[2] != '':
        tmp = options[2].split(',')
        part = part.loc[part['Antigen class'].isin(tmp)]
    if options[3] != '':
        tmp = options[3].split(',')
        part = part.loc[part['Antigen'].isin(tmp)]
    if options[4] != '':
        tmp = options[4].split(',')
        part = part.loc[part['Cell type class'].isin(tmp)]
    if options[5] != '':
        tmp = options[5].split(',')
        part = part.loc[part['Cell type'].isin(tmp)]
    
    part = part.compute()
    part.to_csv("BIGKEK.csv", mode='a', header=False, index=False)
    return tmp

In [90]:
open("BIGKEK.csv", mode = 'w').close()
k = client.map(compute_part, range(df.npartitions), [['', '', '', '', 'Breast,Kidney', '']] * df.npartitions)

In [91]:
[future.result() for future in k]

[['Breast', 'Kidney'], ['Breast', 'Kidney'], ['Breast', 'Kidney']]